In [1]:
import pymongo
import requests
import pandas as pd

from os import getenv
from datetime import datetime
from dotenv import load_dotenv

from config import contract_collection
from helper import contract_list


load_dotenv()

import warnings
warnings.filterwarnings('ignore')

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)

In [2]:
def get_collection_activity(metadata, limit, collection=None, collectionssetid=None, community=None, continuation=None):
    url = 'https://api.reservoir.tools/'
    end_point = 'collections/activity/v5'
    headers = {'accept': '*/*', 'x-api-key': f"{getenv('RESERVOIR')}"}
    if community == None:
        r = requests.get(url + end_point, params=f'?sortBy=eventTimestamp&includeMetadata={metadata}&limit={limit}', headers=headers)
    else:
        r = requests.get(url + end_point, params=f'?sortBy=eventTimestamp&includeMetadata={metadata}&limit={limit}&community={community}', headers=headers)
    data = r.json()
    return data

In [3]:
contract_db = contract_list()

test = get_collection_activity(True, 20, community='artblocks')
new_bids = []
for item in test['activities']:
    if item['type'] == 'bid':
        if item['order']['criteria']['kind'] == 'collection':
            actions = {}
            actions['contract'] = item['contract']
            actions['collection_name'] = item['collection']['collectionName']
            actions['collection_id'] = item['order']['criteria']['data']['collection']['id']
            if item['order']['criteria']['data']['collection']['id'] not in contract_db:
                new_bids.append(actions)

if new_bids:               
    contract_collection().insert_many(new_bids)

contract_db = contract_list()
df = pd.DataFrame(contract_db)

In [4]:
def get_collection_info(limit, topbid, normalize, c_id=None):
    url = 'https://api.reservoir.tools/'
    end_point = 'collections/v5'
    headers = {'accept': '*/*', 'x-api-key': f"{getenv('RESERVOIR')}"}
    r = requests.get(url + end_point, params=f'id={c_id}&includeTopBid={topbid}&normalizeRoyalties={normalize}&sortBy=allTimeVolume&limit={limit}', headers=headers)
    data = r.json()
    return data

In [5]:
new_bids = []
for c in contract_db:
    collections = get_collection_info(1, True, False, c_id=c)
    for collection in collections['collections']:
        project = {}
        id = collection['id']
        project['Collection'] = collection['name']
        contract = collection['primaryContract']
        token_range = collection['tokenSetId'].split(':')[2:]
        project['7day Volume'] = round(collection['volume']['7day'], 2)
        project['Lowest Floor Price'] = f"{str(collection['floorAsk']['sourceDomain'])} : {round(collection['floorAsk']['price']['amount']['decimal'], 3)}"
        try:
            project['Highest Collection Offer'] = f"{str(collection['topBid']['sourceDomain'])} : {round(collection['topBid']['price']['amount']['decimal'], 3)}"
            project['Spread'] = round(float((collection['floorAsk']['price']['amount']['decimal']) - float(collection['topBid']['price']['amount']['decimal'])) / float(collection['floorAsk']['price']['amount']['decimal']), 2)
        except TypeError:
            project['Highest Collection Offer'] = 'None'
            project['Spread'] = 'None'
        new_bids.append(project)
    

df = pd.DataFrame(new_bids)    

In [112]:
df[~df['Spread'].isin(['None'])].sort_values(by='Collection', ascending=True).to_csv('spread.csv', index=False)

In [10]:
get_activity(False, 20, 'desc')

{'activities': [{'id': 1254750014,
   'type': 'bid',
   'contract': '0x092bbc993042a69811d23feb0e64e3bfa0920c6a',
   'collectionId': '0x092bbc993042a69811d23feb0e64e3bfa0920c6a',
   'tokenId': '28685',
   'fromAddress': '0x48063703c49c498878f4be77c29ac1f6e510e1ab',
   'toAddress': None,
   'price': 0.15705,
   'amount': 1,
   'timestamp': 1670448649,
   'order': {'id': '0x26cafa331ec11b650d64219d9b3253fe708182efd250fce6a0682ae5baa4593e'}},
  {'id': 1254750013,
   'type': 'bid',
   'contract': '0x345c2fa23160c63218dfaa25d37269f26c85ca47',
   'collectionId': '0x345c2fa23160c63218dfaa25d37269f26c85ca47',
   'tokenId': '3749',
   'fromAddress': '0xc07003bbd24c2efa3a93d79cac54fac71b1a1a1a',
   'toAddress': None,
   'price': 0.15047,
   'amount': 1,
   'timestamp': 1670448649,
   'order': {'id': '0x3187db7bd3d952c10fa2b82b7b7cfa3ad7b793cf52cb01eb75d8cedc424e496d'}},
  {'id': 1254750012,
   'type': 'bid',
   'contract': '0x740c178e10662bbb050bde257bfa318defe3cabc',
   'collectionId': '0x740c1

In [4]:
url = 'https://api.reservoir.tools/'

response = requests.get(url, json={'limit': 20, 'includeMetadata': False}, headers=headers)

print(response.url)
test = response.json()

https://api.reservoir.tools/activity/v4


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
url = "https://api.reservoir.tools/orders/bids/v5?community=artblocks&includeCriteriaMetadata=false&includeRawData=false&normalizeRoyalties=false&sortBy=createdAt&limit=50"
headers = {
    "accept": "*/*",
    "x-api-key": "demo-api-key"
}

In [2]:
def mongo_collection():
    client = pymongo.MongoClient(getenv('MONGO'))
    db = client.nonfungible 
    return db.globalcontracts

In [5]:
db = mongo_collection()

db.find_one()

{'_id': ObjectId('61bb564679a8467985b2be61'),
 'address': 'simpleassets',
 'isERC20': False,
 'isERC721': True,
 'doSaveTX': True,
 'blockchain': 'wax'}

In [ ]:
start_block = init_collection.find({'blockchain': 'flow'}).sort('blockNumber', -1).limit(1).next()['blockNumber'] - 25